# Game Rental System

This program allows the manager of a game rental service to perform various tasks to assist with management of the service.

## Features

1. **Search Database:** Search for a keyword in the game database by title, genre or platform.
2. **Rent Games:** Rent a copy of a game out to a given customer.
3. **Return Games:** Return a copy of a game that is currently being rented and optionally add feedback for the game.
4. **View Unpopular Games:** Calculates which games are 'unpopular' using various metrics and shows a list of them along with visualisations and advice.
5. **Prune Inventory:** Gives the user the option to remove unpopular games from the database and optionally delete their rental history.

## Instructions

- Click the respective button at the top of the menu to perform a task.

- **Searching the Database:** Enter the keyword to search for in the 'Search by' field and select whether to search by title, genre or platform, then press 'Search'. Leaving the keyword blank will show all entries in the database.
- **Renting a Game:** Enter the ID of the game in the 'Game ID' field and the customer ID in the 'Customer ID' field, then press 'Rent'. Game IDs in the database can be viewed using the search feature, and customer IDs are expected as four lowercase letters.
- **Returning a Game:** Enter the ID of the game in the 'Game ID' field, then press 'Return'.
To return a game with feedback, check the 'Add Feedback' box, use the slider to select a rating and optionally add a comment in the 'Comments' field, then press 'Return'.
- **Viewing Unpopular Games:** Press 'Previous' and 'Next' to cycle through the list of unpopular games. This will display a graph showing the number of times the game has been rented, number of times the game has been reviewed and the average review score for each game. These values will all be shown against the average values across all games. The graph will also show how many days ago the game was purchased by the store, and how many days ago the game was last rented.
Below the graph will be advice generated by the program suggesting whether or not to remove the game based on various factors.
- **Pruning Unpopular Games:** When viewing an unpopular game, the game can be pruned by pressing the 'Prune' button below the 'Previous' and 'Next' buttons. Below the prune button is a checkbox labelled 'Delete Rental History', which allows you to choose whether or not to also delete the game's rental history from the database.

In [1]:
# Last Updated: 12/12/2023

import ipywidgets as widgets
import gameRent as gRent
import gameReturn as gReturn
import gameSearch as gSearch
import inventoryPruning as gPrune

# ----------------------------------------------------------------------
# Global data initialization
# ----------------------------------------------------------------------

page = 0 # Page number used when showing the list of unpopular games
pageID = "" # Game ID of the game occupying this page in the list

# ----------------------------------------------------------------------
# Event handlers
# ----------------------------------------------------------------------

# MENU BUTTON HANDLERS

def SearchClicked(b):
    displayBox = widgets.VBox([searchType,searchInput,btnSubmitSearch])
    with output:
        output.clear_output()
        display(displayBox)

def RentClicked(b):
    displayBox = widgets.VBox([gameIdInput,customerIdInput,btnSubmitRent])
    with output:
        output.clear_output()
        display(displayBox)

def ReturnClicked(b):
    if doFeedbackCheck.value == True:
        displayBox = widgets.VBox([gameIdInput,doFeedbackCheck, ratingSlider,
                               commentInput, btnSubmitReturn]) 
    else:
        displayBox = widgets.VBox([gameIdInput,doFeedbackCheck,
                                   btnSubmitReturn])
    with output:
        output.clear_output()
        display(displayBox)

def UnpopularClicked(b):
    # Get averages and unpopular game information
    averages = gPrune.GetAverages()
    unpopularInfo = gPrune.FindUnpopular(averages)
    unpopularIDs = list(unpopularInfo.keys())
    upSuggestions = gPrune.UnpopularInfo(unpopularInfo, averages)

    # Correct page number if out of range
    global page
    max = len(unpopularIDs) - 1
    if page < 0:
        page = max
    elif page > max:
        page = 0

    # Get the info for the game on the current page and update page header
    gameID = unpopularIDs[page]
    global pageID
    pageID = gameID
    suggestionStr = str(upSuggestions[gameID])
    suggestionList = suggestionStr.split("\n")

    # Build the display
    idLabel = widgets.Label(value=gameID)
    suggestionLabels = [widgets.Label(sug) for sug in suggestionList]
    navBox = widgets.HBox([btnPrev,btnNext])

    displayList = [idLabel]
    displayList = displayList + suggestionLabels
    displayList = displayList + [navBox, btnSubmitPrune, doRemoveRental]
    displayBox = widgets.VBox(displayList)
    with output:
        output.clear_output()
        gPrune.DrawBarChart(gameID,unpopularInfo[gameID],averages)
        display(displayBox)

def PrevClicked(b):
    global page
    page -= 1
    UnpopularClicked(b) # Refresh menu

def NextClicked(b):
    global page
    page += 1
    UnpopularClicked(b) # Refresh menu
        
# ----------------------------------------------------------------------

# SUBMIT BUTTON HANDLERS

def SubmitSearchClicked(b):
    # Search for entries
    column = searchType.value
    item = searchInput.value
    results = gSearch.searchGames(column, item)
    if results == None:
        with output:
            output.clear_output()
            print("No results found")
            return

    # Output all entry info
    headers = ["Game ID", "Platform", "Genre", "Title",
               "Publisher", "Purchase Date", "Status"]
    infoLabels = [widgets.Label(header) for header in headers]
    for i in range (0, len(results)):
        newInfo = [widgets.Label(stat) for stat in results[i]]
        infoLabels = infoLabels + newInfo
    boxLayout = widgets.Layout(grid_template_columns="repeat(7, 150px)")
    displayBox = widgets.GridBox(infoLabels, layout=boxLayout)
    with output:
        output.clear_output()
        display(displayBox)
    
def SubmitRentClicked(b):
    renterID = customerIdInput.value
    gameID = gameIdInput.value
    status = gRent.RentGame(renterID, gameID)
    with output:
        output.clear_output()
        print(status)

def SubmitReturnClicked(b):
    gameID = gameIdInput.value
    status = gReturn.ReturnGame(gameID)
    # Add feedback if selected and return was successful
    if doFeedbackCheck.value == True and status[0] == "R":
        rating = ratingSlider.value
        comment = commentInput.value
        s = gReturn.AddFeedback(gameID, rating, comment)
        # Don't add invalid game ID error to prevent displaying it twice
        if s[-1] != "D":
            status = status + "\n" + s
    with output:
        output.clear_output()
        print(status)

def SubmitPruneClicked(b):
    gameID = pageID
    removeRental = doRemoveRental.value
    status = gPrune.PruneGame(gameID, removeRental)
    with output:
        output.clear_output()
        print(status)

In [2]:
# ----------------------------------------------------------------------
# GUI Layout
# ----------------------------------------------------------------------

# Buttons for each menu option
btnSearch = widgets.Button(description="Search Games")
btnRent = widgets.Button(description="Rent Game")
btnReturn = widgets.Button(description="Return Game")
btnUnpopular = widgets.Button(description="Unpopular Games")
# Submit buttons for each menu
btnSubmitSearch = widgets.Button(description="Search")
btnSubmitRent = widgets.Button(description="Rent")
btnSubmitReturn = widgets.Button(description="Return")
btnSubmitPrune = widgets.Button(description="Prune")

# Input boxes for game ID, customer ID and search bar
gameIdInput = widgets.Text(description="Game ID:")
customerIdInput = widgets.Text(description="Customer ID:")
searchInput = widgets.Text(description="Search for:")

# Radio buttons for search type
searchType = widgets.RadioButtons(
    options=['Title', 'Genre', 'Platform'],
    description='Search by:',
)

# Extra inputs for collecting feedback
doFeedbackCheck = widgets.Checkbox(description="Add Feedback")
ratingSlider = widgets.IntSlider(description="Rating:",min=1,max=5)
commentInput = widgets.Text(description="Comments:")

# Menu layout for unpopular games tab
btnPrev = widgets.Button(description="Previous")
btnNext = widgets.Button(description="Next")
doRemoveRental = widgets.Checkbox(description = "Delete Rental History")

# Output widget
output = widgets.Output()

# Link menu buttons to event handlers
btnSearch.on_click(SearchClicked)
btnRent.on_click(RentClicked)
btnReturn.on_click(ReturnClicked)
btnUnpopular.on_click(UnpopularClicked)
doFeedbackCheck.observe(ReturnClicked)
btnPrev.on_click(PrevClicked)
btnNext.on_click(NextClicked)
# Link submit buttons to event handlers
btnSubmitSearch.on_click(SubmitSearchClicked)
btnSubmitRent.on_click(SubmitRentClicked)
btnSubmitReturn.on_click(SubmitReturnClicked)
btnSubmitPrune.on_click(SubmitPruneClicked)

tabs = widgets.Box([btnSearch, btnRent, btnReturn, btnUnpopular])
display(tabs, output)

Box(children=(Button(description='Search Games', style=ButtonStyle()), Button(description='Rent Game', style=B…

Output()